# Collecting US decennial census data

In this notebook, we'll collect demographic data from the US decennial census, by county.

The census website has an API, which is good, because everything else about the census website is close to unusable.
The api is described here: http://www.census.gov/data/developers/data-sets/decennial-census-data.html

As a quick demonstration, we can use the API to get population data for every county in the US:

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('http://api.census.gov/data/2010/sf1?get=P0120001&for=county:*')
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.head()

,P0120001,state,county
1,54571,01,001
2,182265,01,003
3,27457,01,005
4,22915,01,007
5,57322,01,009


The census code descriptions can also be accessed via the API. A listing of the field names is available here: http://api.census.gov/data/2010/sf1/variables.html

In [4]:
pd.read_json('http://api.census.gov/data/2010/sf1/variables/P0120001.json', typ='ser')

concept    P12. Sex By Age [49]
label          Total population
name                   P0120001
dtype: object

## Collect data on male population by age, county

For now I'm only going to look at males. This is probably a bad idea in general.


### Start with the 2010 census
The male population is broken down into some somewhat arbitrary cohorts, each with its own name. We want all of the fields between `P0120003` and `P0120025`.

We'll do some data munging to get it in numeric format, and to take care of the labels and indicies.


In [5]:
fields = ['P01200%02i'%i for i in range(3,26)]
url = 'http://api.census.gov/data/2010/sf1?get=%s&for=county:*'%','.join(fields)
print url
pops2010 = pd.read_json(url)
pops2010.columns = pops2010.iloc[0]
pops2010.drop(pops2010.index[0], inplace=True)
pops2010 = pops2010.applymap(float)
pops2010.set_index(['state', 'county'], inplace=True)
pops2010.head()

http://api.census.gov/data/2010/sf1?get=P0120003,P0120004,P0120005,P0120006,P0120007,P0120008,P0120009,P0120010,P0120011,P0120012,P0120013,P0120014,P0120015,P0120016,P0120017,P0120018,P0120019,P0120020,P0120021,P0120022,P0120023,P0120024,P0120025&for=county:*


0             P0120003  P0120004  P0120005  P0120006  P0120007  P0120008  \
state county                                                               
1     1           1866      2001      2171      1417       796       350   
      3           5614      5832      6076      3704      2226      1013   
      5            847       826       820       559       360       190   
      7            712       759       771       513       293       122   
      9           1805      1936      2113      1340       799       340   

0             P0120009  P0120010  P0120011  P0120012    ...     P0120016  \
state county                                            ...                
1     1            279       910      1543      1594    ...         1866   
      3            862      2918      5183      5317    ...         6425   
      5            192       666      1212      1162    ...         1000   
      7            167       522       987      1013    ...          847   
      9            294       943      1735      1730    ...         1972   

0             P0120017  P0120018  P0120019  P0120020  P0120021  P0120022  \
state county                                                               
1     1           1524       494       785       418       596       807   
      3           5943      2301      3427      2054      2841      3663   
      5            910       358       501       280       351       436   
      7            734       294       390       188       268       347   
      9           1810       716       984       546       806      1039   

0             P0120023  P0120024  P0120025  
state county                                
1     1            546       295       159  
      3           2644      1735      1176  
      5            303       195       129  
      7            232       138        73  
      9            684       418       234  

[5 rows x 23 columns]

### Get data from 2000
The 2000 census (logically) has different codes for its data, and (even more logically) breaks the cohorts down differently. In this case, we can get data for each age year with codes `PCT012003` through `PCT012104`. The api limits us to only 50 columns at a time, so we'll do it in chunks and stitch them together.

In [6]:
fields = ['PCT012%03i'%i for i in range(3,105)]

dflist = []
chunkSize = 40
for i in range(0, len(fields), chunkSize):
    chunk = fields[i:i+chunkSize]
    url = 'http://api.census.gov/data/2000/sf1?get=%s&for=county:*'%','.join(chunk)
    print url
    df_chunk = pd.read_json(url)
    df_chunk.columns = df_chunk.iloc[0]
    df_chunk.drop(df_chunk.index[0], inplace=True)
    df_chunk = df_chunk.applymap(float)
    df_chunk.set_index(['state', 'county'], inplace=True)
    dflist.append(df_chunk)

pops2000 = pd.concat(dflist,axis=1)
pops2000 = pops2000.applymap(float)
pops2000.head()

http://api.census.gov/data/2000/sf1?get=PCT012003,PCT012004,PCT012005,PCT012006,PCT012007,PCT012008,PCT012009,PCT012010,PCT012011,PCT012012,PCT012013,PCT012014,PCT012015,PCT012016,PCT012017,PCT012018,PCT012019,PCT012020,PCT012021,PCT012022,PCT012023,PCT012024,PCT012025,PCT012026,PCT012027,PCT012028,PCT012029,PCT012030,PCT012031,PCT012032,PCT012033,PCT012034,PCT012035,PCT012036,PCT012037,PCT012038,PCT012039,PCT012040,PCT012041,PCT012042&for=county:*
http://api.census.gov/data/2000/sf1?get=PCT012043,PCT012044,PCT012045,PCT012046,PCT012047,PCT012048,PCT012049,PCT012050,PCT012051,PCT012052,PCT012053,PCT012054,PCT012055,PCT012056,PCT012057,PCT012058,PCT012059,PCT012060,PCT012061,PCT012062,PCT012063,PCT012064,PCT012065,PCT012066,PCT012067,PCT012068,PCT012069,PCT012070,PCT012071,PCT012072,PCT012073,PCT012074,PCT012075,PCT012076,PCT012077,PCT012078,PCT012079,PCT012080,PCT012081,PCT012082&for=county:*
http://api.census.gov/data/2000/sf1?get=PCT012083,PCT012084,PCT012085,PCT012086,PCT012087,PCT0

0             PCT012003  PCT012004  PCT012005  PCT012006  PCT012007  \
state county                                                          
1     1             264        305        293        331        309   
      3             877        865        845        873        926   
      5             185        184        196        173        191   
      7             179        146        150        145        157   
      9             344        347        374        394        376   

0             PCT012008  PCT012009  PCT012010  PCT012011  PCT012012  \
state county                                                          
1     1             364        342        374        382        411   
      3             856        951        981       1031       1118   
      5             223        187        236        193        234   
      7             148        183        140        147        151   
      9             377        400        361        402        366   

0               ...      PCT012095  PCT012096  PCT012097  PCT012098  \
state county    ...                                                   
1     1         ...              6          2          0          7   
      3         ...             31         28         14         13   
      5         ...              3          3          5          3   
      7         ...              6          2          2          2   
      9         ...              6          9          1          6   

0             PCT012099  PCT012100  PCT012101  PCT012102  PCT012103  PCT012104  
state county                                                                    
1     1               2          0          0          3          1          0  
      3               2          5          6          4          5          0  
      5               2          2          1          0          0          0  
      7               3          2          0          0          0          0  
      9               0          3          0          3          5          0  

[5 rows x 102 columns]

## Align the datasets
As they have different cohorts, we need to do some summation before we can merge the two census years into a single table. I'll break the data down into 10-year cohorts by selecting columns to stitch together. We'll set breakpoints by the last few digits of the field name, and label our new cohorts according to which decade of your life they are. We're using 1-based indexing here for the cohort names.

In [7]:
pops2010d = pd.DataFrame(index=pops2010.index)

decades = ['dec_%i'%i for i in range(1,10)]
breakpoints_2010 = [3, 5, 8, 12, 14, 16, 18, 22, 24, 26]
for dec, s, f in zip(decades, breakpoints_2010[:-1], breakpoints_2010[1:]):
    pops2010d[dec] = pops2010[['P0120%03i'%i for i in range(s,f)]].sum(axis=1)
    
pops2010d.head()

dec_1  dec_2  dec_3  dec_4  dec_5  dec_6  dec_7  dec_8  dec_9
state county                                                               
1     1        3867   4384   3082   3598   4148   3390   2293   1353    454
      3       11446  12006   9976  11042  12517  12368  10623   6307   2911
      5        1673   1739   2260   2208   2233   1910   1490    739    324
      7        1471   1577   1798   2016   1928   1581   1140    579    211
      9        3741   4252   3312   3719   4129   3782   3052   1723    652

In [8]:
pops2000d = pd.DataFrame(index=pops2000.index)

decades = ['dec_%i'%i for i in range(1,10)]
breakpoints_2000 = [3, 13, 23, 33, 43, 53, 63, 73, 83, 104]
for dec, s, f in zip(decades, breakpoints_2000[:-1], breakpoints_2000[1:]):
    pops2000d[dec] = pops2000[['PCT012%03i'%i for i in range(s,f)]].sum(axis=1)

pops2000d.head()

dec_1  dec_2  dec_3  dec_4  dec_5  dec_6  dec_7  dec_8  dec_9
state county                                                               
1     1        3375   3630   2461   3407   3283   2319   1637    825    284
      3        9323  10094   7600   9725  10379   8519   6675   4711   1822
      5        2002   2198   2412   2465   2178   1699   1026    689    301
      7        1546   1460   1680   1762   1624   1237    774    475    187
      9        3741   3615   3393   3901   3773   3007   2227   1269    550

Now that the data have been formatted in the same way, we'll concatenate them. We also drop any rows that don't show up in both datasets.

In [13]:
frame = pd.concat([pops2000d, pops2010d], keys=[2000, 2010], axis=1)
frame.dropna(inplace=True)
frame.head()

2000                                                     2010  \
             dec_1  dec_2 dec_3 dec_4  dec_5 dec_6 dec_7 dec_8 dec_9  dec_1   
state county                                                                  
1     1       3375   3630  2461  3407   3283  2319  1637   825   284   3867   
      3       9323  10094  7600  9725  10379  8519  6675  4711  1822  11446   
      5       2002   2198  2412  2465   2178  1699  1026   689   301   1673   
      7       1546   1460  1680  1762   1624  1237   774   475   187   1471   
      9       3741   3615  3393  3901   3773  3007  2227  1269   550   3741   

                                                                   
              dec_2 dec_3  dec_4  dec_5  dec_6  dec_7 dec_8 dec_9  
state county                                                       
1     1        4384  3082   3598   4148   3390   2293  1353   454  
      3       12006  9976  11042  12517  12368  10623  6307  2911  
      5        1739  2260   2208   2233   1910   1490   739   324  
      7        1577  1798   2016   1928   1581   1140   579   211  
      9        4252  3312   3719   4129   3782   3052  1723   652

I'm happy with this format, so we'll save it to csv:

In [15]:
frame.to_csv('Males by decade and county.csv')

As our dataframe has a [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/advanced.html) we have to take care when re-importing from the csv to get the index and header columns correct.

In [16]:
pd.read_csv('Males by decade and county.csv', header=[0,1], index_col=[0,1])

2000                                                        \
              dec_1  dec_2  dec_3  dec_4  dec_5  dec_6  dec_7 dec_8 dec_9   
state county                                                                
1     1        3375   3630   2461   3407   3283   2319   1637   825   284   
      3        9323  10094   7600   9725  10379   8519   6675  4711  1822   
      5        2002   2198   2412   2465   2178   1699   1026   689   301   
      7        1546   1460   1680   1762   1624   1237    774   475   187   
      9        3741   3615   3393   3901   3773   3007   2227  1269   550   
      11        840    885   1059    952    968    628    390   238   179   
      13       1497   1766   1076   1219   1496   1199    832   599   334   
      15       7212   8032   7544   7613   8237   6544   4535  2846  1137   
      17       2543   2540   2185   2437   2504   2088   1474   992   522   
      19       1552   1506   1424   1664   1726   1598   1322   717   285   
      21       2957   2904   2656   2872   2882   2317   1612  1002   379   
      23       1097   1164    839    956   1080   1010    710   430   203   
      25       2130   2247   1496   1785   1904   1521   1120   654   323   
      27        939   1025    854    992    983    844    664   432   217   
      29        965   1031    879   1061   1048    906    643   349   155   
      31       2903   3246   2986   3006   3067   2628   1826  1189   452   
      33       3606   3773   3156   3740   3939   3310   2458  1632   697   
      35       1008   1069    762    838    914    865    579   458   174   
      37        813    872    814   1005    966    712    529   387   134   
      39       2403   2608   2068   2458   2643   2211   1809  1237   555   
      41        921   1026    766    807    944    826    549   431   193   
      43       5230   5602   5008   5695   5596   4595   3351  2230   907   
      45       3860   3656   3712   3650   3446   2659   1806  1162   405   
      47       3477   3901   2486   2587   3044   2317   1658  1121   501   
      49       4615   4491   4493   4697   4571   3768   2540  1663   670   
      51       4684   4939   4945   5530   5172   3825   2361  1336   548   
      53       2559   2757   2949   2955   3078   2224   1585   927   441   
      55       6842   7225   6406   6824   7328   6187   4167  3261  1266   
      57       1199   1363   1071   1213   1310   1162    847   536   234   
      59       2108   2303   2255   2221   2091   1758   1391   871   330   
...             ...    ...    ...    ...    ...    ...    ...   ...   ...   
55    129       911   1301    661   1017   1256   1107    891   615   312   
      131      8537   9141   6305   9977   9934   6906   4025  2589  1194   
      133     24815  27971  17718  26947  32031  22825  13003  8520  3653   
      135      3451   4069   2595   3789   4098   2996   2080  1821  1000   
      137      1373   1784    977   1510   1812   1456   1396   992   369   
      139     10095  11708  12046  12620  12521   8297   5210  3777  1875   
      141      5065   5916   4018   5515   5946   4268   2874  2215  1213   
56    1        1666   2803   4997   1859   2005   1555    848   545   251   
      3         860   1032    514    639    817    713    558   403   199   
      5        2728   3245   2306   2475   3515   1823    748   366   102   
      7         985   1206   1027   1221   1523   1143    666   414   191   
      9         900   1043    557    788   1118    772    441   286   102   
      11        358    532    246    345    502    420    334   163    79   
      13       2539   3099   1872   2187   2880   2181   1511  1061   410   
      15        816   1022    674    716    974    779    576   442   235   
      17        246    359    187    248    400    338    273   200    97   
      19        419    540    291    375    566    510    365   292   118   
      21       5759   6176   6018   6631   6274   4698   2751  1878   801   
      23   